# Inference and Evaluation (Baseline Model)

Performs inference using the baseline model on the test set for comparison.

---
**Paper Reference:** *Evaluation of nnU-Net for FCD II Lesion Segmentation in FLAIR MRI*

# Baseline FLAIR Testing & Evaluation


## 1. Setup & Installation


In [ ]:
!pip install -q nnunetv2

## 2. Environment Configuration


In [ ]:
import os
from pathlib import Path
import shutil, nibabel as nib, json, pandas as pd, numpy as np
from scipy.spatial.distance import directed_hausdorff

# USER CONFIG
DATASET_ID = 2  # <--- ID of the dataset to use
DATASET_NAME = "BonnFCD_FLAIR"  # <--- Name of the dataset
CFG = "3d_fullres"  # <--- nnU-Net configuration (e.g., 2d, 3d_fullres)
FOLD = "0"   # <--- Fold number to use (0, 1, 2, 3, 4)
CKPT = "checkpoint_best.pth"  # <--- Checkpoint file name (e.g., checkpoint_best.pth)
TRAINER = "nnUNetTrainer"  # Your custom trainer name

DATASET_DIR = f"Dataset{DATASET_ID:03d}_{DATASET_NAME}"
INPUT_ROOT = Path("/kaggle/input/flair-training-baseline-dataset-1")  # <--- Root directory for input data
RAW_SRC = INPUT_ROOT / "nnUNet_raw_data" / DATASET_DIR

# nnU-Net environment variables
os.environ["nnUNet_raw"] = str(INPUT_ROOT / "nnUNet_raw_data")
os.environ["nnUNet_preprocessed"] = str(INPUT_ROOT / "nnUNet_preprocessed")
os.environ["nnUNet_results"] = str(INPUT_ROOT / "nnUNet_results")

print(f"✅ Environment configured for dataset: {DATASET_DIR}")
print(f"📁 Input root: {INPUT_ROOT}")
print(f"📁 Raw data: {RAW_SRC}")

# Use imagesTs and labelsTs directly from INPUT_ROOT
IMAGES_TS = RAW_SRC / "imagesTs"
LABELS_TS = RAW_SRC / "labelsTs"

print(f"📁 Test images: {IMAGES_TS}")
print(f"📁 Test labels: {LABELS_TS}")

# Verify folders exist
if not IMAGES_TS.exists():
    raise FileNotFoundError(f"imagesTs folder not found at {IMAGES_TS}")
if not LABELS_TS.exists():
    raise FileNotFoundError(f"labelsTs folder not found at {LABELS_TS}")

print(f"✅ Found {len(list(IMAGES_TS.glob('*.nii*')))} images")
print(f"✅ Found {len(list(LABELS_TS.glob('*.nii*')))} labels")

## 3. Inference


In [ ]:
INP = IMAGES_TS
OUT = Path(f"/kaggle/working/preds_D{DATASET_ID:03d}_{CFG}_fold{FOLD}_best")
OUT.mkdir(parents=True, exist_ok=True)

print("🔍 Running inference ...")
print(f"   Input: {INP}")
print(f"   Output: {OUT}")
print(f"   Trainer: {TRAINER}")
print(f"   Fold: {FOLD}")

!nnUNetv2_predict -i "$INP" -o "$OUT" -d "$DATASET_ID" -c "$CFG" -f "$FOLD" -tr "$TRAINER" -chk "$CKPT"
print(f"✅ Predictions saved to: {OUT}")

## 4. Evaluation Metrics


In [ ]:
def dice_score(gt, pred):
    intersection = np.sum((gt > 0) & (pred > 0))
    total = np.sum(gt > 0) + np.sum(pred > 0)
    return 2 * intersection / total if total > 0 else 1.0

def iou_score(gt, pred):
    intersection = np.sum((gt > 0) & (pred > 0))
    union = np.sum((gt > 0) | (pred > 0))
    return intersection / union if union > 0 else 1.0

def precision_recall_f1(gt, pred):
    tp = np.sum((gt > 0) & (pred > 0))
    fp = np.sum((gt == 0) & (pred > 0))
    fn = np.sum((gt > 0) & (pred == 0))
    precision = tp / (tp + fp) if (tp + fp) > 0 else 1.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 1.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

results = []
print(f"\n🔎 Looking for labels in: {LABELS_TS}")
print(f"🔎 Looking for predictions in: {OUT}")

label_files = list(sorted(LABELS_TS.glob("*.nii*")))
pred_files = list(sorted(OUT.glob("*.nii*")))

print(f"\n📋 Found {len(label_files)} label files")
print(f"📋 Found {len(pred_files)} prediction files")

if len(label_files) == 0:
    print("❌ ERROR: No label files found!")
if len(pred_files) == 0:
    print("❌ ERROR: No prediction files found!")

for lbl_file in label_files:
    stem = lbl_file.stem.replace('.nii', '')  # Handle both .nii and .nii.gz

    # Try to find matching prediction with different extensions
    pred_file = None
    for ext in ['.nii', '.nii.gz']:
        potential_pred = OUT / f"{stem}{ext}"
        if potential_pred.exists():
            pred_file = potential_pred
            break

    if pred_file is None:
        print(f"⚠️  No prediction found for {stem}")
        continue

    print(f"✓ Processing {stem}")
    gt = nib.load(str(lbl_file)).get_fdata()
    pred = nib.load(str(pred_file)).get_fdata()

    dice = dice_score(gt, pred)
    iou = iou_score(gt, pred)
    prec, rec, f1 = precision_recall_f1(gt, pred)

    results.append({
        "Case": stem,
        "Dice": dice,
        "IoU": iou,
    })

## 5. Results Analysis


In [ ]:
if len(results) == 0:
    print("\n❌ ERROR: No results computed. Check that predictions match labels.")
else:
    df = pd.DataFrame(results).round(4)
    df.to_csv("/kaggle/working/full_metrics_per_case.csv", index=False)

    print(f"\n{df.to_string()}")
    print("\n" + "="*60)
    print("📊 SUMMARY STATISTICS")
    print("="*60)
    print(f"📊 Mean Dice:       {df['Dice'].mean():.4f}")
    print(f"📊 Mean IoU:        {df['IoU'].mean():.4f}")
    # print(f"🎯 Mean Precision:  {df['Precision'].mean():.4f}")
    # print(f"🎯 Mean Recall:     {df['Recall'].mean():.4f}")
    # print(f"🏅 Mean F1-score:   {df['F1'].mean():.4f}")
    print("="*60)
    print("💾 Saved → /kaggle/working/full_metrics_per_case.csv")